# LMStudio Model Comparison Tool

Test user-provided text blocks across multiple LMStudio models.

## Models to Test:
1. Mistral 7B Instruct v0.3
2. Meta-Llama-3-8B-Instruct
3. Nous Hermes 2 Mistral 7B
4. GPT-OSS 20B
5. Qwen2 7B Instruct
6. Phi-3 Mini 3.8B Instruct

In [16]:
import asyncio
import httpx
import json
import os
import time
from datetime import datetime
from typing import Dict, List, Optional, Tuple
from IPython.display import display, HTML, Markdown
import pandas as pd

# Configuration
LMSTUDIO_BASE_URL = os.getenv("LMSTUDIO_API_URL", "http://localhost:1234/v1")
if "/v1" not in LMSTUDIO_BASE_URL:
    LMSTUDIO_BASE_URL = f"{LMSTUDIO_BASE_URL.rstrip('/')}/v1"

# Models to test (exact names as they appear in LMStudio)
# Update these to match your actual model names
MODELS = [
    "mistral-7b-instruct-v0.3",
    "Meta-Llama-3-8B-Instruct",
    "Nous-Hermes-2-Mistral-7B",
    "GPT-OSS-20B",
    "Qwen2-7B-Instruct",
    "Phi-3-mini-3.8B-Instruct"
]

print(f"LMStudio URL: {LMSTUDIO_BASE_URL}")
print(f"Models to test: {len(MODELS)}")

LMStudio URL: http://localhost:1234/v1
Models to test: 6


## Step 1: Check Available Models

Verify which models are currently loaded in LMStudio.

In [21]:
async def get_available_models() -> List[str]:
    """Fetch list of available models from LMStudio."""
    try:
        async with httpx.AsyncClient() as client:
            response = await client.get(
                f"{LMSTUDIO_BASE_URL}/models",
                timeout=10.0
            )
            if response.status_code == 200:
                data = response.json()
                models = [model["id"] for model in data.get("data", [])]
                return models
            else:
                print(f"❌ Failed to fetch models: {response.status_code}")
                return []
    except Exception as e:
        print(f"❌ Error fetching models: {e}")
        return []

# Get available models
available_models = await get_available_models()

print(f"\n✅ Available models in LMStudio ({len(available_models)}):")
for model in available_models:
    print(f"  - {model}")

# Filter models that are actually available
valid_models = []
for model in MODELS:
    # Try to match (case-insensitive, partial match)
    matched = False
    for avail in available_models:
        if model.lower().replace("-", "") in avail.lower().replace("-", "") or \
           avail.lower().replace("-", "") in model.lower().replace("-", ""):
            valid_models.append((model, avail))
            matched = True
            break
    if not matched:
        print(f"⚠️  Model '{model}' not found in LMStudio")

print(f"\n📋 Models to test: {len(valid_models)}")
for original, actual in valid_models:
    print(f"  {original} → {actual}")


✅ Available models in LMStudio (14):
  - nous-hermes-2-mistral-7b-dpo
  - phi-3-mini-3.8b-instructiontuned-alpaca
  - qwen2-7b-instruct
  - openai/gpt-oss-20b
  - mistralai/mistral-7b-instruct-v0.3
  - text-embedding-e5-base-v2
  - meta/llama-3.3-70b
  - llama-3.2-1b-instruct
  - text-embedding-nomic-embed-text-v1.5
  - meta-llama-3.1-8b-instruct
  - codellama-7b-instruct
  - stable-code-instruct-3b
  - phi-2
  - meta-llama-3-8b-instruct

📋 Models to test: 6
  mistral-7b-instruct-v0.3 → mistralai/mistral-7b-instruct-v0.3
  Meta-Llama-3-8B-Instruct → meta-llama-3-8b-instruct
  Nous-Hermes-2-Mistral-7B → nous-hermes-2-mistral-7b-dpo
  GPT-OSS-20B → openai/gpt-oss-20b
  Qwen2-7B-Instruct → qwen2-7b-instruct
  Phi-3-mini-3.8B-Instruct → phi-3-mini-3.8b-instructiontuned-alpaca


## Step 2: Set Your Test Text

**Edit the cell below** to provide the text block you want to test across all models.

In [22]:
# EDIT THIS: Your test text
TEST_TEXT = r"""
 Generate a Sigma detection rule based on this threat intelligence:

**Article Title:** Crypto24 Ransomware Uncovered: Stealth, Persistence, and Enterprise-Scale Impact
**Source:** Picus Security Blog
**URL:** https://www.picussecurity.com/resource/blog/crypto24-ransomware-uncovered-stealth-persistence-and-enterprise-scale-impact

**SIGMA Rule Requirements:**
1. **Focus on TTPs:** Create rules for Tactics, Techniques, and Procedures, not atomic IOCs
2. **Behavioral Patterns:** Detect attack behaviors and techniques
3. **Proper Syntax:** Use correct SIGMA rule format and structure
4. **Required Fields:** Include all mandatory SIGMA fields (title, description, logsource, detection, etc.)
5. **Actionable Detection:** Rules should detect real attack patterns
6. **MITRE ATT&CK Tags:** Extract specific ATT&CK technique IDs from the content (e.g., T1059.001 for PowerShell execution, T1486 for Data Encrypted for Impact/Ransomware)

**Rule Guidelines:**
- Avoid rules for single IP addresses or file hashes
- Focus on command-line patterns, process creation, and system modifications
- Use appropriate log sources (Windows Event Logs, Sysmon, Linux auditd, etc.)
- Include proper categorization and references
- Make rules specific enough to avoid false positives

**CRITICAL OUTPUT REQUIREMENTS:**
- Output ONLY the YAML rule content
- Do NOT include markdown code blocks (```yaml or ```)
- Do NOT include any explanatory text before or after the rule
- Start directly with the YAML content

**YAML Formatting Rules:**
- Use exactly 2 spaces for each indentation level
- NEVER use tabs
- Ensure proper YAML syntax
- All field names must be lowercase
- Use quotes for values containing special characters

**Required YAML Structure:**
title: <Short Title in Title Case>
id: <UUIDv4>
status: experimental
description: <What this rule detects and why it matters>
author: <Name, Name>
date: YYYY/MM/DD
modified: YYYY/MM/DD
references:
  - <link-1>
tags:
  - attack.<tactic>
  - attack.t####[.###]
  - car.####-##-###
  - cve.####-#####
logsource:
  category: <process_creation|network_connection|...>
  product: <windows|linux|macos>
detection:
  selection:
    <FieldName>: <Value or List>
  condition: selection
fields:
  - <key-field-1>
falsepositives:
  - <expected-benign-case>
level: <informational|low|medium|high|critical>

**Threat Intelligence Content:**
n for stealth, precision, and adaptability, making its campaigns especially impactful. Analyzed Malware Samples of Crypto24 Ransomware Samples by Picus Labs SHA256: 3b0b4a11ad576588bae809ebb546b4d985ef9f37ed335ca5e2ba6b886d997bac SHA256: 686bb5ee371733ab7908c2f3ea1ee76791080f3a4e61afe8b97c2a57fbc2efac SHA256 : 24f7b66c88ba085d77c5bd386c0a0ac3b78793c0e47819a0576b60a67adc7b73 In this blog, we are going to examine the group s tactics, evasion techniques, and the implications for defenders. Valid Accounts Crypto24 leveraged account manipulation to maintain privileged access. Attackers reactivated dormant default administrator accounts and created new accounts with generic names to avoid raising suspicion. Using net.exe , they modified and re-enabled accounts, then added them to the Administrators group for elevated privileges. net user /add net user administrator /active:yes net user net localgroup administrators /add net localgroup "Remote Desktop Users" /add This dual strategy of reusing ors group for elevated privileges. net user /add net user administrator /active:yes net user net localgroup administrators /add net localgroup "Remote Desktop Users" /add This dual strategy of reusing built-in accounts and introducing fresh ones ensured redundant privileged access, reduced detection likelihood, and enabled reliable persistence for lateral movement and ransomware deployment. Discovery The actor conducted reconnaissance through two distinct command executions. System Profiling via Batch Script #Process cmd.exe /c '\ \Scan\1.bat' wmic partition get name,size, type enumerated disk partitions wmic computersystem get TotalPhysicalMemory,caption collected memory and OS details. This allowed the actor to build a profile of system hardware and storage configuration. Account and Privilege Enumeration # Process cmd.exe net user listed local accounts net localgroup identified group memberships. vilege Enumeration # Process cmd.exe net user listed local accounts net localgroup identified group memberships. This step provided visibility into existing access and privilege levels, helping the actor identify potential targets for lateral movement. By combining system and account reconnaissance, the attacker gained a comprehensive understanding of the host environment, enabling effective planning for subsequent attack stages. Persistence Crypto24 ransomware actors established persistence through scheduled tasks, malicious services, and privileged account creation, disguising activity within legitimate Windows processes to reduce detection. Scheduled Tasks # Process wscript.exe / cmd.exe %ProgramData%\Update\update.vbs executed at regular intervals %ProgramData%\Update\vm.bat executed at regular intervals These tasks ensured periodic execution of malicious payloads to maintain a foothold. Malicious Services # Process sc. t regular intervals These tasks ensured periodic execution of malicious payloads to maintain a foothold. Malicious Services # Process sc.exe (Service Control) Keylogger: sc create WinMainSvc type = share start= auto binPath= "C:\Windows\System32\scvhost.exe -k WinMainSvc" Ransomware: sc create MSRuntime type = share start= auto binpath= "C:\Windows\System32\svchost.exe -k MSRuntime" displayname= "Microsoft Runtime Manager" The attacker leveraged svchost.exe to masquerade as legitimate services, enabling stealth deployment of a keylogger and the Crypto24 ransomware. Account Creation # Process update.bat Crypto24 ransomware holders created a new user account and added it to Administrators and Remote Desktop Users groups. This granted elevated privileges and remote access, further strengthening persistence. Through recurring tasks, disguised services, and privileged accounts, the attacker ensured durable and covert access to compromised systems. ng persistence. Through recurring tasks, disguised services, and privileged accounts, the attacker ensured durable and covert access to compromised systems. Privilege Escalation The threat actor escalated privileges using a combination of batch scripts, runas.exe , and PsExec, granting administrative rights and enabling high-level system access. Administrator Group Modification # Process cmd.exe /c C:\update.bat net.exe localgroup administrators john /add net.exe localgroup administrators service.lot9 /add net.exe localgroup administrators 00025436 /add net.exe localgroup "administrators" "NetUser" /add net.exe localgroup "Remote Desktop Users" "NetUser" /add net.exe localgroup administrators IT.Guest /add These commands added multiple accounts to the Administrators and Remote Desktop Users groups, granting them elevated privileges and remote access capabilities. Runas Execution # Process C:\Windows\explorer.exe runas. s groups, granting them elevated privileges and remote access capabilities. Runas Execution # Process C:\Windows\explorer.exe runas.exe /user:administrator cmd This opened a Command Prompt session with administrator-level privileges under the specified account. PsExec Execution # Process $mytemp $\low\psexec64.exe -u -p cmd PsExec was leveraged to launch a command prompt using privileged user credentials, enabling remote privilege escalation and administrative control. Through user group modification, impersonation with runas.exe , and remote execution with PsExec, the actor ensured persistent administrative access and expanded control over compromised systems. Defense Evasion To maintain access and avoid detection, the actor repeatedly created administrative accounts, initiated RDP sessions, and leveraged custom tools to weaken host defenses. A notable component was the deployment of a tool resembling RealBlindingEDR , observed across multiple endpoints, specifically designed to bypas ns, and leveraged custom tools to weaken host defenses. A notable component was the deployment of a tool resembling RealBlindingEDR , observed across multiple endpoints, specifically designed to bypass Endpoint Detection and Response (EDR) mechanisms. Tool Deployment The following files were identified. %USERPROFILE%. \AppData\Local\Temp\Low\ AVB.exe %USERPROFILE%. \AppData\Local\Temp\Low\ AVMon.exe %PROGRAMDATA%\update\ avb.exe Command-Line Arguments The tool accepts a single argument to specify driver number and associated function. 1 Loads WdFilter.sys (Windows Defender Filter Driver) 2 Loads MpKslDrv.sys (Microsoft Malware Protection Kernel Service Library Driver) 3 Loads mpsdrv.sys (Microsoft Protection Service Driver) 4 Loads WdNisDrv.sys (Windows Defender Network Inspection Service Driver) This behavior directly targeted core Microsoft security drivers to impair defense capabilities. 4); if ( result ) break ; v4 = &SubStr[++v5]; if ( !v4 ) return result; } return ( char *) 1 ; By dynamically querying driver metadata and filtering against a vendor list, the tool selectively disabled protections from well-known security products. This allowed the attacker to maintain a persistent presence on compromised hosts while evading endpoint detection and incident response mechanisms. Lateral Movement: Remote Services Following initial compromise and account creation, the threat actor expanded their control using PsExec, registry modifications, and firewall rules to enable Remote Desktop Protocol (RDP) access. They also deployed a network scanning utility to identify additional systems in the environment for lateral movement. PsExec Service Deployment # Process C:\Windows\ PSEXESVC.exe This binary is installed and executed by PsExec when initiating remote sessions. It allowed the attacker to run commands interactively on the compromised system under the context of privileged c C.exe This binary is installed and executed by PsExec when initiating remote sessions. It allowed the attacker to run commands interactively on the compromised system under the context of privileged credentials. By establishing PsExec service execution, the actor ensured reliable remote administration capabilities and a channel for pushing additional commands or payloads. Registry Modification to Enable RDP # Process C:\Windows\System32\ cmd.exe # Command C:\Windows\System32\ reg.exe ADD "HKEY_LOCAL_MACHINE\System\CurrentControlSet\Control\Terminal Server" /v fDenyTSConnections /t REG_DWORD /d 0 /f This command directly modified the registry key fDenyTSConnections , setting its value to 0 . By doing so, the attacker explicitly enabled RDP connections on the host, which are normally disabled in hardened environments. This change provided a persistent mechanism for graphical remote access, complementing the use of PsExec. Firewall Rule Addition # Process C:\Windows\System32\ netsh. d in hardened environments. This change provided a persistent mechanism for graphical remote access, complementing the use of PsExec. Firewall Rule Addition # Process C:\Windows\System32\ netsh.exe # Command advfirewall firewall add rule name= "Open Remote Desktop" protocol=TCP dir= in localport=3389 action=allow After enabling RDP in the registry, the attacker ensured accessibility by configuring Windows Firewall to allow inbound traffic on TCP port 3389. This effectively exposed the host to external RDP sessions, bypassing local network restrictions and strengthening their lateral movement strategy. Network Scanning Utility # Process C:\Program Files\Google\Chrome\Application\ chrome.exe # Command $myuserprofile $\downloads\ipscan-3.9.1- setup.exe The actor installed Advanced IP Scanner (version 3.9.1) on one endpoint. This tool is widely used to enumerate active devices, open ports, and reachable services within a local network. (version 3.9.1) on one endpoint. This tool is

[Prompt truncated to fit model context window]
"""

# Optional: System prompt
SYSTEM_PROMPT = "You are a cybersecurity analyst. Provide clear, concise analysis."

# Optional: Generation parameters
MAX_TOKENS = 8000
TEMPERATURE = 0.2

print(f"Test text length: {len(TEST_TEXT)} characters")
print(f"\nPreview:\n{TEST_TEXT[:200]}...")

Test text length: 12051 characters

Preview:

 Generate a Sigma detection rule based on this threat intelligence:

**Article Title:** Crypto24 Ransomware Uncovered: Stealth, Persistence, and Enterprise-Scale Impact
**Source:** Picus Security Blo...


## Step 3: Run Test Across All Models

This will query each model sequentially and display results.

In [19]:
async def query_model(
    model_name: str,
    user_text: str,
    system_prompt: Optional[str] = None,
    max_tokens: int = 2000,
    temperature: float = 0.3
) -> Tuple[str, Dict, float, Optional[str]]:
    """
    Query a single model and return results.
    
    Returns:
        (model_name, response_data, duration_seconds, error_message)
    """
    start_time = time.time()
    
    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_text})
    
    payload = {
        "model": model_name,
        "messages": messages,
        "max_tokens": max_tokens,
        "temperature": temperature
    }
    
    try:
        async with httpx.AsyncClient() as client:
            response = await client.post(
                f"{LMSTUDIO_BASE_URL}/chat/completions",
                headers={"Content-Type": "application/json"},
                json=payload,
                timeout=300.0  # 5 minute timeout
            )
            
            duration = time.time() - start_time
            
            if response.status_code == 200:
                result = response.json()
                content = result.get('choices', [{}])[0].get('message', {}).get('content', '')
                
                # Extract usage stats if available
                usage = result.get('usage', {})
                
                return (model_name, {
                    'content': content,
                    'usage': usage,
                    'full_response': result
                }, duration, None)
            else:
                error_msg = f"HTTP {response.status_code}: {response.text[:200]}"
                return (model_name, {}, time.time() - start_time, error_msg)
                
    except httpx.TimeoutException:
        return (model_name, {}, time.time() - start_time, "Request timeout (>300s)")
    except httpx.ConnectError:
        return (model_name, {}, time.time() - start_time, "Connection error - is LMStudio running?")
    except Exception as e:
        return (model_name, {}, time.time() - start_time, f"Error: {str(e)}")

In [20]:
# Run tests sequentially (to avoid overloading LMStudio)
results = []

print("🚀 Starting model comparison tests...\n")
print(f"Test time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
print("=" * 80)

for idx, (original_name, actual_model_name) in enumerate(valid_models, 1):
    print(f"\n[{idx}/{len(valid_models)}] Testing: {original_name}")
    print(f"   Model ID: {actual_model_name}")
    print(f"   Status: ", end="", flush=True)
    
    model_name, response_data, duration, error = await query_model(
        actual_model_name,
        TEST_TEXT,
        SYSTEM_PROMPT if SYSTEM_PROMPT.strip() else None,
        MAX_TOKENS,
        TEMPERATURE
    )
    
    if error:
        print(f"❌ Failed ({duration:.2f}s): {error}")
        results.append({
            'model': original_name,
            'model_id': actual_model_name,
            'status': 'error',
            'duration': duration,
            'error': error,
            'response': None,
            'tokens': None
        })
    else:
        content = response_data.get('content', '')
        usage = response_data.get('usage', {})
        tokens = usage.get('total_tokens', 'N/A')
        
        print(f"✅ Success ({duration:.2f}s, {tokens} tokens)")
        
        results.append({
            'model': original_name,
            'model_id': actual_model_name,
            'status': 'success',
            'duration': duration,
            'error': None,
            'response': content,
            'tokens': tokens,
            'usage': usage
        })

print("\n" + "=" * 80)
print(f"\n✅ Testing complete! {len([r for r in results if r['status'] == 'success'])}/{len(results)} models succeeded")

🚀 Starting model comparison tests...

Test time: 2025-10-30 18:26:31


[1/6] Testing: mistral-7b-instruct-v0.3
   Model ID: mistralai/mistral-7b-instruct-v0.3
   Status: ❌ Failed (3.78s): HTTP 400: {"error":"Error rendering prompt with jinja template: \"Only user and assistant roles are supported!\".\n\nThis is usually an issue with the model's prompt template. If you are using a popular model, 

[2/6] Testing: Meta-Llama-3-8B-Instruct
   Model ID: meta-llama-3-8b-instruct
   Status: ✅ Success (22.76s, 3273 tokens)

[3/6] Testing: Nous-Hermes-2-Mistral-7B
   Model ID: nous-hermes-2-mistral-7b-dpo
   Status: ✅ Success (52.20s, 4962 tokens)

[4/6] Testing: GPT-OSS-20B
   Model ID: openai/gpt-oss-20b
   Status: ✅ Success (61.46s, 4723 tokens)

[5/6] Testing: Qwen2-7B-Instruct
   Model ID: qwen2-7b-instruct
   Status: ✅ Success (17.61s, 2965 tokens)

[6/6] Testing: Phi-3-mini-3.8B-Instruct
   Model ID: phi-3-mini-3.8b-instructiontuned-alpaca
   Status: ✅ Success (163.82s, 11116 tokens)


✅

## Step 4: View Results Summary

In [11]:
# Summary table
summary_data = []
for r in results:
    summary_data.append({
        'Model': r['model'],
        'Status': '✅' if r['status'] == 'success' else '❌',
        'Duration (s)': f"{r['duration']:.2f}" if r['duration'] else 'N/A',
        'Tokens': r['tokens'] if r['tokens'] != 'N/A' else 'N/A',
        'Error': r['error'][:50] + '...' if r['error'] else ''
    })

df_summary = pd.DataFrame(summary_data)
display(df_summary)

,Model,Status,Duration (s),Tokens,Error
0,mistral-7b-instruct-v0.3,❌,2.46,NaN,"HTTP 400: {""error"":""Error rendering prompt wit..."
1,Meta-Llama-3-8B-Instruct,✅,22.43,3314.0,
2,Nous-Hermes-2-Mistral-7B,✅,26.23,3844.0,
3,GPT-OSS-20B,✅,57.66,4437.0,
4,Qwen2-7B-Instruct,✅,42.27,2930.0,
5,Phi-3-mini-3.8B-Instruct,✅,177.28,11118.0,


## Step 5: Detailed Responses

View full responses from each model.

In [12]:
# Display detailed responses
for r in results:
    print("\n" + "=" * 80)
    print(f"\n🔹 {r['model']} ({r['model_id']})")
    print(f"Status: {'✅ Success' if r['status'] == 'success' else '❌ Failed'}")
    print(f"Duration: {r['duration']:.2f}s")
    
    if r['status'] == 'success':
        if r['tokens']:
            print(f"Tokens: {r['tokens']}")
        print(f"\nResponse:\n")
        print(r['response'])
    else:
        print(f"\nError: {r['error']}")
    
    print("\n" + "-" * 80)



🔹 mistral-7b-instruct-v0.3 (mistralai/mistral-7b-instruct-v0.3)
Status: ❌ Failed
Duration: 2.46s

Error: HTTP 400: {"error":"Error rendering prompt with jinja template: \"Only user and assistant roles are supported!\".\n\nThis is usually an issue with the model's prompt template. If you are using a popular model, 

--------------------------------------------------------------------------------


🔹 Meta-Llama-3-8B-Instruct (meta-llama-3-8b-instruct)
Status: ✅ Success
Duration: 22.43s
Tokens: 3314

Response:

Here is the Sigma detection rule based on the provided threat intelligence:

```yaml
title: Crypto24 Ransomware Detection Rule
id: c5c4a6d8-7f3b-4e1a-bf2c-9a0a5c5e5c5e
status: experimental
description: Detects tactics, techniques, and procedures (TTPs) used by Crypto24 ransomware.
author: [Your Name]
date: 2023/02/15
modified: 2023/03/01
references:
  - https://www.picussecurity.com/resource/blog/crypto24-ransomware-uncovered-stealth-persistence-and-enterprise-scale-impact
tags:


## Step 6: Side-by-Side Comparison

Compare responses in a structured HTML table.

In [13]:
# Create HTML comparison table
html_output = "<h2>Model Response Comparison</h2>\n"
html_output += f"<p><strong>Test Text:</strong><br><pre>{TEST_TEXT[:200]}...</pre></p>\n"
html_output += "<table border='1' cellpadding='10' style='border-collapse: collapse; width: 100%;'>\n"
html_output += "<tr><th>Model</th><th>Status</th><th>Duration</th><th>Tokens</th><th>Response Preview</th></tr>\n"

for r in results:
    status_icon = '✅' if r['status'] == 'success' else '❌'
    preview = r['response'][:200] + '...' if r['response'] else r['error'] or 'N/A'
    preview = preview.replace('\n', '<br>').replace('<', '&lt;').replace('>', '&gt;')
    
    html_output += f"<tr>\n"
    html_output += f"  <td><strong>{r['model']}</strong><br><small>{r['model_id']}</small></td>\n"
    html_output += f"  <td>{status_icon}</td>\n"
    html_output += f"  <td>{r['duration']:.2f}s</td>\n"
    html_output += f"  <td>{r['tokens'] if r['tokens'] != 'N/A' else 'N/A'}</td>\n"
    html_output += f"  <td style='max-width: 400px; word-wrap: break-word;'>{preview}</td>\n"
    html_output += f"</tr>\n"

html_output += "</table>"

display(HTML(html_output))

Model,Status,Duration,Tokens,Response Preview
mistral-7b-instruct-v0.3mistralai/mistral-7b-instruct-v0.3,❌,2.46s,None,"HTTP 400: {""error"":""Error rendering prompt with jinja template: \""Only user and assistant roles are supported!\"".\n\nThis is usually an issue with the model's prompt template. If you are using a popular model,"
Meta-Llama-3-8B-Instructmeta-llama-3-8b-instruct,✅,22.43s,3314,Here is the Sigma detection rule based on the provided threat intelligence:<br><br>```yaml<br>title: Crypto24 Ransomware Detection Rule<br>id: c5c4a6d8-7f3b-4e1a-bf2c-9a0a5c5e5c5e<br>status: experimental<br>description...
Nous-Hermes-2-Mistral-7Bnous-hermes-2-mistral-7b-dpo,✅,26.23s,3844,---<br>title: Crypto24 Ransomware Stealth and Persistence Detection<br>id: 65a0f78b-3d9e-41c1-9f2a-1234567890ab<br>status: experimental<br>description: This rule detects behaviors associated with the Crypto24 ran...
GPT-OSS-20Bopenai/gpt-oss-20b,✅,57.66s,4437,title: Account Creation and Group Modification via Net.exe<br>id: d6f5c7a0-9e12-4b1a-8f3c-2a1b3c4d5e6f<br>status: experimental<br>description: Detects creation of local user accounts and addition to privileged...
Qwen2-7B-Instructqwen2-7b-instruct,✅,42.27s,2930,```yaml<br>title: Crypto24 Ransomware Persistence Tactics<br>id: 0000-0000-0000-0000<br>status: experimental<br>description: Detects persistence and evasion techniques used by the Crypto24 ransomware.<br>author: Pic...
Phi-3-mini-3.8B-Instructphi-3-mini-3.8b-instructiontuned-alpaca,✅,177.28s,11118,"## for the task with an analysis: Wencthacking-intultion - AttsniNostiliationsnate, The following description and <br> a/forespace (Inc.10sideswise(MikIfthektype to beicoteskizlater-data for the end-HARE..."


## Step 7: Export Results

Save results to JSON for later analysis.

In [ ]:
# Export to JSON
export_data = {
    'test_timestamp': datetime.now().isoformat(),
    'test_text': TEST_TEXT,
    'system_prompt': SYSTEM_PROMPT,
    'parameters': {
        'max_tokens': MAX_TOKENS,
        'temperature': TEMPERATURE
    },
    'results': results
}

output_file = f"lmstudio_comparison_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_file, 'w') as f:
    json.dump(export_data, f, indent=2)

print(f"✅ Results exported to: {output_file}")

# Export to CSV format
import csv

csv_file = f"lmstudio_comparison_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    # Header row
    writer.writerow(['Duration (s)', 'Model Name', 'Output', 'Temperature'])
    
    # Data rows
    for r in results:
        duration = f"{r['duration']:.2f}" if r['duration'] else 'N/A'
        model_name = r['model']
        output = r['response'] if r['response'] else (r['error'] or 'N/A')
        # Replace newlines with spaces for CSV compatibility
        output_clean = output.replace('\n', ' ').replace('\r', ' ')
        temperature = TEMPERATURE
        
        writer.writerow([duration, model_name, output_clean, temperature])

print(f"✅ CSV exported to: {csv_file}")
print(f"   Rows: {len(results)}")
print(f"\nPreview of CSV file:")
print(f"  Columns: Duration (s), Model Name, Output, Temperature")

In [ ]:
# Export to CSV format
import csv

csv_file = f"lmstudio_comparison_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

with open(csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    
    # Header row
    writer.writerow(['Duration (s)', 'Model Name', 'Output', 'Temperature'])
    
    # Data rows
    for r in results:
        duration = f"{r['duration']:.2f}" if r['duration'] else 'N/A'
        model_name = r['model']
        output = r['response'] if r['response'] else (r['error'] or 'N/A')
        # Replace newlines with spaces for CSV compatibility
        output_clean = output.replace('\n', ' ').replace('\r', ' ')
        temperature = TEMPERATURE
        
        writer.writerow([duration, model_name, output_clean, temperature])

print(f"✅ CSV exported to: {csv_file}")
print(f"   Rows: {len(results)}")